In [54]:
import pandas as pd
import sys
sys.path.insert(1, '/home/juan/Documents/')
import time
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from credentials import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [69]:
database = pd.read_csv('database.csv')

In [58]:
def search_song():
    title = str(input('Please enter song title: ')).lower()
    artist = str(input('Please enter artist name: ')).lower()
    try:
        results = sp.search(q=title+','+artist, limit=5)
        song_id = results['tracks']['items'][-1]['id']
        return song_id
    except: 
        return 'na'

In [49]:
#search_song()

In [44]:
def get_audio_features(song_id):
    my_dict = {}
    los=[]
    try:
        x = sp.audio_features(song_id)[0]
        if x: los.append(x)
    except:
        pass
    df = pd.DataFrame(los)
    df = df[['danceability','energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
    return df

In [92]:
def scaler(df):
    X = df.copy()
    scaler = StandardScaler()
    scaler.fit(X)
    X_scaled = scaler.transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
    return X_scaled_df

In [ ]:
def clustering():
    kmeans = KMeans(n_clusters=7, random_state=1234)
    kmeans.fit(X_scaled_df)
    clusters = kmeans.predict(X_scaled_df)
    pd.Series(clusters).value_counts().sort_index()
    



In [103]:
def song_reccomender():
    #Get song id
    song_id = search_song()
    
    #Get audio features of the song and display them
    df = get_audio_features(song_id)
    display(df)
    
    #Scale song feat by adding it to our database and then returning it scaled.
    database = pd.read_csv('database.csv')
    df2 = database[['danceability','energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
    df3 = pd.concat([df2,df],axis=0)
    df_scaled = scaler(df3)
    
    
    
    
    song_tr = df_scaled.iloc[[-1]]
    
    
    return song_tr

In [104]:
song_reccomender()

Please enter song title:  back in black
Please enter artist name:  acdc


,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.324,0.95,0.0774,0.000593,0.000001,0.987,0.511,92.703


,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo
3095,-1.924988,1.673372,-0.377883,-0.879697,-0.452582,5.368577,0.204182,-0.965717


In [95]:
type(df3)

pandas.core.frame.DataFrame

In [96]:
df3

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.207880,0.967394,-0.757843,-0.751047,0.456588,-0.387429,0.195759,-0.723313
1,0.509132,-1.378124,-0.553759,0.752547,2.602779,-0.440912,-1.307716,-0.778204
2,-0.117473,-1.196985,-0.717192,1.155143,-0.450291,0.073853,-0.275919,-0.826363
3,-0.352450,0.228904,-0.733784,0.657015,-0.452586,0.187503,1.168596,-0.992041
4,-0.250024,-0.286645,-0.558737,-0.506419,-0.452586,-0.180186,-0.031657,0.412349
...,...,...,...,...,...,...,...,...
3091,-2.045489,-0.997268,-0.732125,2.178692,-0.452586,-0.220298,-0.928689,-0.473909
3092,-1.250183,-2.575036,-0.605195,2.444815,2.710933,-0.487708,-1.413001,-0.525686
3093,0.292230,0.214971,-0.436785,-0.880263,0.078047,-0.869437,-0.406473,0.618517
3094,1.226113,-1.080870,-0.057654,-0.879584,0.892586,-0.719018,0.949603,-0.319284
